In [ ]:
#[0]이전실험, 데이터 재수령(200711)

from e3fp.pipeline import confs_from_smiles
from e3fp.pipeline import fprints_from_mol
from e3fp.pipeline import mol_to_sdf

In [ ]:
#Q1) 최종 제출할 데이터 CSV 파일 만들어 여기서 불러오기 1.Active, 2.Inactive, 3. 3rd
import pandas as pd
df1 = pd.read_csv("NO inhibit(Final).CSV", encoding='cp949')
df = pd.read_csv("dataset_cleaned.csv") 

In [ ]:
#[1]제공된 Conf분리
from rdkit import *
from rdkit.Chem.EState import Fingerprinter
from rdkit.Chem import *
from rdkit.Chem.rdmolops import RDKFingerprint
import pandas as pd
import numpy as np
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem.Draw import IPythonConsole
IPythonConsole.ipython_useSVG=True

In [ ]:
#Q2) 바뀐 최종 제출 데이터에 맞춰서 코드 수정  및 실행

#Read the data
filename = 'NO_Inhibit'
data = pd.read_csv(filename + '.csv')
names = data['Num'].tolist()
compound = data['Synonym'].tolist()

#Add some new columns
data['Mol'] = data['SMILES'].apply(Chem.MolFromSmiles)
mols = data['Mol'].tolist()
output = {}
prefix = [str(x) for x in range(0, len(data))]
mols = [ mol for mol in Chem.SDMolSupplier( filename + '.sdf', removeHs=True ) ]
for i  in range (0, len(data)):
    mols[i].SetProp("_Name", str(names[i]))
    mols[i].SetProp("Synonym", str(compound[i]))
    #output[i]=Chem.SDWriter('['+prefix[i]+']'+ names[i]+'.sdf')
    output[i]=Chem.SDWriter(str(names[i])+'.sdf')
    output[i].write(mols[i])
    output[i].close()

In [ ]:
#[3]Active와 Inactive나누기
import glob
files = glob.glob( '*.sdf' )

with open( 'Inactive.sdf', 'w' ) as result:
    for file_ in files:
        for line in open( file_, 'r' ):
            result.write( line )     

In [ ]:
import glob
files = glob.glob( '*.sdf' )

with open( 'Active.sdf', 'w' ) as result:
    for file_ in files:
        for line in open( file_, 'r' ):
            result.write( line )

In [ ]:
import glob
files = glob.glob( '*.sdf' )

with open( '3rd-set.sdf', 'w' ) as result:
    for file_ in files:
        for line in open( file_, 'r' ):
            result.write( line )

In [ ]:
#[2]Omega돌리기 - code는 omgega 결과물의 FP 생성임
from e3fp.fingerprint.generate import fp, fprints_dict_from_mol, fprints_dict_from_sdf
from e3fp.conformer.generate import generate_conformers
from e3fp.pipeline import fprints_from_sdf, fprints_from_mol
from e3fp.fingerprint.fprint import save, savez
from e3fp.fingerprint.db import FingerprintDatabase
from e3fp.fingerprint.fprint import Fingerprint

In [ ]:
#Q3)  1.Active, 2.Inactive, 3. 3rd - 3개에 맞춰 수정

# mols = [ mol for mol in Chem.SDMolSupplier("Inactive.sdf", removeHs=True ) ]
active = [ mol for mol in Chem.SDMolSupplier("Active.sdf") ]
inactive = [ mol for mol in Chem.SDMolSupplier("Inactive.sdf") ]
third = [ mol for mol in Chem.SDMolSupplier("3rd-set.sdf") ]

lens(active)
lens(inactive)
lens(third)

# generate E3FP fingerprints and collect original molecule information
activefp = [fprints_dict_from_mol( mol) for mol in active ]
inactivefp = [fprints_dict_from_mol( mol) for mol in inactive ]
thirdfp = [fprints_dict_from_mol( mol) for mol in third ]
# fpdicts = [fprints_dict_from_mol(mol) for mol in mols ]

binary = list()



In [ ]:
#Q3)  1.Active, 2.Inactive, 3. 3rd - 3개에 맞춰 수정
#Parameter:
#fp_type(type)–Type of fingerprint (Fingerprint, CountFingerprint, FloatFingerprint).
#level(int)–Level, or number of iterations used during fingerprinting.
#name(str)–Name of database

db1 = FingerprintDatabase(fp_type = Fingerprint, level = 5, name = "Active")
db2 = FingerprintDatabase(fp_type = Fingerprint, level = 5, name = "Inactive")
db3 = FingerprintDatabase(fp_type = Fingerprint, level = 5, name = "3rd-set")
print(len(db1))
print(len(db2))
print(len(db3))

In [ ]:
#Q3)  1.Active, 2.Inactive, 3. 3rd - 3개에 맞춰 수정

on_inds  = [np.random. uniform(0, 2**32, size =30) for i in range (len(activefp))]
fps  = [Fingerprint(x, bits = 2**32) for x in on_inds]

#Parameter:
#indices(array_like of int)–log2(bits)-bit indices in a sparse bitvector of bits which correspond to 1.
#bits(int)–Number of bits in bitvector.
#level(int)–Level of fingerprint, corresponding to fingerprinting iterations.
#name(str)–Name of fingerprint.
#props(dict)–Custom properties of fingerprint, consisting of a string keyword and some value.

for i in range(len(activefp)):
    db1.add_fingerprints(activefp[i][5])
db1.savez('Active')

In [ ]:
# define input and output files
sdf_file = "Active.sdf"
e3fp_file = "Active.fps"

# generate E3FP fingerprints and collect original molecule information
fps = []
mol_info = []
with open(sdf_file, "rb") as f:
    suppl = Chem.ForwardSDMolSupplier(f)
    for mol in suppl:
        if mol is not None:
            fpdicts = fprints_dict_from_mol(mol)
            fps.append(fpdicts)
            mol_info.append(mol)

In [ ]:
# create fingerprint database and save fingerprints and original molecule information

for fp_dict, mol in zip(fps,mol_info):
    for fp_key, fp_val in fp_dict.items():
        entry = db1.set_prop(fp_key, fp_val)
        entry.SetMol(mol)        
        db1.add_fingerprints(fp_val)
db1.savez(e3fp_file)

In [ ]:
# [5]GenFPBitLoc
#Read Smiles List, Generate Fingerprint Database

def find(s, ch):
    return [i for i, ltr in enumerate(s) if ltr == ch]

def generate_Loclist(db, data):
    fpname = db.fp_names
    binfp1 = [fp.fold().to_rdkit() for fp in db]
    str1 = [fp.ToBitString() for fp in binfp1]
    loclist = []

    for i in range(len(str1)):
        loclist.append(np.array(find(str1[i], '1')))
        
    data = data.astype({'Num': str})
    
    templist = []
    for i in range(len(fpname)):
        templist.append(fpname[i].split('_')[0])
        
    result = pd.DataFrame([fpname, loclist, templist]).T
    result.columns = ['conf', 'Loc', 'Num']
    result2 = pd.merge(result, data)
    result2.to_csv('FPLoc.csv')
    return result2

In [ ]:
data = pd.read_csv('NO_Inhibit_Labeled.csv')

fdb1 = FingerprintDatabase.load("Active.fps.bz2")
fdb2 = FingerprintDatabase.load("Inactive.fps.bz2")

result_Active = generate_Loclist(fdb1, data)
result_InActive = generate_Loclist(fdb2, data)
pd.concat([result_Active, result_InActive]).to_csv('dataset.csv')